In [ ]:
import helpers as hp
# import Preprocessing - Separating Recipes as prep
from preprocess import pos_tag_db, lemmatize_db
import nltk
from itertools import permutations, chain
from collections import Counter
from nltk.metrics import BigramAssocMeasures
from math import log

In [ ]:
# co occurrences 

# we need to add a function that changes the pos-tag of all ingredients to 'ingre' instead of nouns

spansize = 5 # symmetric ±5w span

cooccs_surface = Counter()

for recipe in lemmatized: # lemmatized? not sure it's the right one
    for i,w in enumerate(recipe):
        if w.split("-")[-1] in ["ingre"]: # ingredients is the target vectors 
            span_range = list(range(max(i - spansize, 0), i)) # left side indices (range, then list so we can extend)
            span_range.extend(range(i + 1, min(i + spansize + 1, len(recipe)))) # extend by right side indices
            for cw in [recipe[idx] for idx in span_range]:
                cooccs_surface[(w, cw)] += 1

In [ ]:
cooccs_surface.most_common(20) # print 20 most common co-occurrences

In [ ]:
# filtering least and most frequent lemmas 

# least frequent
lemmas_frequencies = Counter(chain(*lemmatized))
print("least common lemmas:", list(reversed(lemmas_frequencies.most_common()))[:20])

# most frequent 
print("most common lemmas:", lemmas_frequencies.most_common(20))

In [ ]:
filtered_lemmas_frequencies = Counter()
selected_ingre_frequencies = Counter()

for k,v in lemmas_frequencies.items():
    pos = k.split("-")[-1]
    if v >= 10 and \
    pos in ["NN", "NNS", "NNP", "NNPS", "JJ", "JJR", "JJS", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "IN", "CD", "ingre"]
        filtered_lemmas_frequencies[k] = v
        if pos == "ingre" : # targeting all ingredients
            selected_ingre_frequencies[k] = v

print(">>", len(filtered_lemmas_frequencies), "context lemmas")
print(filtered_lemmas_frequencies.most_common(20))
print(">>", len(selected_ingre_frequencies), "target nouns")

In [ ]:
# filtered co-occurrences

filtered_cooccs_surface = Counter()

for k,v in cooccs_surface.items():
    if k[0] in selected_ingre_frequencies and k[1] in filtered_lemmas_frequencies:
        filtered_cooccs_surface[k] = v

In [ ]:
filtered_cooccs_surface.most_common(20) # print 20 filteres most common co-occurrences

In [ ]:
# vector representations 

# Measure of choice - sppmi

def sppmi(o_11, r_1, c_1, all_freqs_sum, n):
    """Smoothed Positive Pointwise Mutual Information"""
    return ppmi(o_11, r_1, (c_1**0.75)/all_freqs_sum, n)

sppmi_surface = Counter()

N = sum(cooccs_surface.values())  # note that this is NOT the reduced dictionary
s = sum([v**0.75 for v in reuters_lemmas_frequencies.values()]) # calculating Pa(c)

for k,v in filtered_cooccs_surface.items():
    sppmi_surface[k] = sppmi(v, lemmas_frequencies[k[0]], lemmas_frequencies[k[1]], s, N) 

In [ ]:
sppmi_surface.most_common(15) 